In [1]:
# !pip install -r ../requirements.txt

In [2]:
!pip install essentia 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 4.7 MB/s eta 0:00:0000:0100:01


In [1]:
import pandas as pd
import numpy as np
import ast
import json
import os
from essentia.standard import MonoLoader

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [2]:
from tqdm.notebook import tqdm

In [3]:
tqdm.pandas()

In [4]:

args = pd.Series({
    "root_dir":"/mnt/disks/data/",
    "dataset_path":"/mnt/disks/data/fma/fma_large"
})


In [5]:


base_path = os.path.join(args.root_dir,"fma")


# In[17]:

metadata_path_fma = os.path.join(base_path,"fma_metadata")



In [6]:
# Cria um dicionário que associa o ID de cada música aos IDs de seus gêneros musicais
tracks_df = pd.read_csv(os.path.join(metadata_path_fma,'tracks.csv'), header=[0,1], index_col=0)

In [7]:
tracks_df.reset_index(inplace=True)

In [8]:
tracks_df.columns = ['_'.join(col) for col in tracks_df.columns.values]

In [9]:
tracks_df.columns

Index(['track_id_', 'album_comments', 'album_date_created',
       'album_date_released', 'album_engineer', 'album_favorites', 'album_id',
       'album_information', 'album_listens', 'album_producer', 'album_tags',
       'album_title', 'album_tracks', 'album_type', 'artist_active_year_begin',
       'artist_active_year_end', 'artist_associated_labels', 'artist_bio',
       'artist_comments', 'artist_date_created', 'artist_favorites',
       'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_members', 'artist_name', 'artist_related_projects',
       'artist_tags', 'artist_website', 'artist_wikipedia_page', 'set_split',
       'set_subset', 'track_bit_rate', 'track_comments', 'track_composer',
       'track_date_created', 'track_date_recorded', 'track_duration',
       'track_favorites', 'track_genre_top', 'track_genres',
       'track_genres_all', 'track_information', 'track_interest',
       'track_language_code', 'track_license', 'track_listens',
 

In [10]:
tracks_df = tracks_df[['track_genres_all','track_genres','track_id_']]

In [11]:
tracks_df.reset_index(inplace=True)

In [12]:
tracks_df.dropna(inplace=True)

In [13]:
tracks_df.track_genres_all.value_counts()

track_genres_all
[21]                     2735
[15]                     2689
[]                       2231
[12]                     1896
[1, 38, 41, 247, 30]     1633
                         ... 
[32, 1, 38, 41, 250]        1
[32, 38, 103, 10, 17]       1
[10, 5]                     1
[10, 539, 21]               1
[169, 10, 12, 9]            1
Name: count, Length: 4151, dtype: int64

In [14]:
tracks_df["track_genres_all"] = tracks_df.track_genres_all.apply(lambda x : ast.literal_eval(x))

In [15]:
# Remover linhas com genre_id vazio
tracks_df = tracks_df[tracks_df['track_genres_all'].map(len) > 0]


In [16]:
tracks_df

,index,track_genres_all,track_genres,track_id_
0,0,[21],[21],2
1,1,[21],[21],3
2,2,[21],[21],5
3,3,[10],[10],10
4,4,"[17, 10, 76, 103]","[76, 103]",20
...,...,...,...,...
106569,106569,"[25, 12]",[25],155316
106570,106570,"[25, 12]",[25],155317
106571,106571,"[25, 12]",[25],155318
106572,106572,"[25, 12]",[25],155319


In [17]:
def find_path(track_id, dataset_path):
    track_id = track_id.zfill(6)
    folder_id = track_id[0:3]
    file_path = os.path.join(dataset_path, folder_id, track_id+'.mp3')
    return file_path
# In[36]:


tracks_df['file_path'] = tracks_df.track_id_.apply(lambda x: find_path(str(x), args.dataset_path))



In [18]:
## Fazer o valid genre aqui

In [19]:
def valid_music(file_path):
    try:
        # we start by instantiating the audio loader:
        loader = MonoLoader(filename=file_path)
    
        return True
    except:
        return False

In [20]:
valid_music('/mnt/disks/data/fma/fma_large/000/000003.mp3')

True

In [21]:
tracks_df.loc[:,'valid'] = tracks_df.file_path.progress_apply(lambda x: valid_music(x))

  0%|          | 0/104343 [00:00<?, ?it/s]

In [22]:
tracks_df_filter = tracks_df[tracks_df['valid'] == True]

In [23]:
def find_path(track_id,dataset_path):
    track_id = track_id.zfill(6)
    folder_id = track_id[0:3]
    file_path = os.path.join(dataset_path,folder_id,track_id+'.mp3')
    return file_path

In [24]:
tracks_df_filter['file_path'] = tracks_df_filter.track_id_.apply(lambda x: find_path(str(x),args.dataset_path))

/tmp/ipykernel_148978/3260058484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df_filter['file_path'] = tracks_df_filter.track_id_.apply(lambda x: find_path(str(x),args.dataset_path))


In [25]:
tracks_df_filter.iloc[0].file_path

'/mnt/disks/data/fma/fma_large/000/000002.mp3'

In [26]:
tracks_df_filter = tracks_df_filter[['track_genres_all','track_genres','track_id_','file_path']]

In [27]:
tracks_df_filter.to_csv(os.path.join(metadata_path_fma,'tracks_valid.csv'),index=False)